# Project 2. HMM 적용하여 데이터 모델링 해보기 (자유주제)

## Hidden Markov Model

* 은닉마코프모델 계산 및 구현
  * https://ratsgo.github.io/machine%20learning/2017/10/14/computeHMMs/
* https://web.stanford.edu/~jurafsky/slp3/A.pdf
  
## ~~Taxi Service Trajectory (TST)~~
* Taxi Service Trajectory (TST) Prediction Challenge 2015
  * http://www.geolink.pt/ecmlpkdd2015-challenge/index.html
  * Artificial Neural Networks Applied to Taxi Destination Prediction
    * https://arxiv.org/pdf/1508.00021.pdf
  
## Human Activity Recognition (HAR)
* ~~Smartphone Dataset for Human Activity Recognition (HAR) in Ambient Assisted Living (AAL) Data Set~~
  * [Dataset](https://archive.ics.uci.edu/ml/datasets/Smartphone+Dataset+for+Human+Activity+Recognition+%28HAR%29+in+Ambient+Assisted+Living+%28AAL%29)
  * Author
    * *Kadian Alicia Davis, Evans Boateng Owusu* 
  * Structure
    * Triaxial acceleration from the accelerometer (total acceleration)
      * `final_acc_train.txt`, `final_acc_test.txt`
    * Triaxial Angular velocity from the gyroscope. 
      * `final_gyro_train.txt`, `final_gyro_test.txt` 
    * A 561-feature vector with time and frequency domain variables 
      * `final_X_train.txt`, `final_X_test.txt`
    * The corresponding activity labels
      * `final_y_train.txt`, `final_y_test.txt`
* **Human Activity Recognition Using Smartphones Data Set**
  * [Dataset](https://archive.ics.uci.edu/ml/datasets/Human+Activity+Recognition+Using+Smartphones)
  * Author
    * *Jorge L. Reyes-Ortiz, Davide Anguita, Alessandro Ghio, Luca Oneto and Xavier Parra*
  * Structure
    * Raw Data
      * `acc_exp#{1~61}_user{1~30}.txt`
      * `gyro_exp#{1~61}_user{1~30}.txt`
      * `labels.txt`

In [1]:
from scipy import io
from hmmlearn import hmm

import matplotlib.pyplot as plt
import numpy as np
import time

labels = np.loadtxt('./HAR/RawData/labels.txt', delimiter=' ', dtype=int)
N = len(labels)

acc = np.empty((N), dtype=object)
gyro = np.empty((N), dtype=object)

y = np.empty((N),dtype=int)

actions =['WALKING', 'WALKING_UPSTAIRS', 'WALKING_DOWNSTAIRS', 'SITTING',
          'STANDING', 'LAYING', 'STAND_TO_SIT', 'SIT_TO_STAND', 'SIT_TO_LIE', 
          'LIE_TO_SIT', 'STAND_TO_LIE', 'LIE_TO_STAND']
num_actions = len(actions)

In [2]:
prev = ''
acc_file = np.empty([])
gyro_file = np.empty([])
for (i, row) in enumerate(labels):
    filename = 'exp{:02d}_user{:02d}.txt'.format(row[0], row[1])
    
    if prev != filename:
        acc_file = np.loadtxt('./HAR/RawData/acc_' + filename, delimiter=' ')
        gyro_file = np.loadtxt('./HAR/RawData/gyro_' + filename, delimiter=' ')
        prev = filename
        print(filename)
    
    acc[i] = acc_file[row[3]:row[4]+1, :]
    gyro[i] = gyro_file[row[3]:row[4]+1, :]
    y[i] = row[2]-1

exp01_user01.txt
exp02_user01.txt
exp03_user02.txt
exp04_user02.txt
exp05_user03.txt
exp06_user03.txt
exp07_user04.txt
exp08_user04.txt
exp09_user05.txt
exp10_user05.txt
exp11_user06.txt
exp12_user06.txt
exp13_user07.txt
exp14_user07.txt
exp15_user08.txt
exp16_user08.txt
exp17_user09.txt
exp18_user09.txt
exp19_user10.txt
exp20_user10.txt
exp21_user10.txt
exp22_user11.txt
exp23_user11.txt
exp24_user12.txt
exp25_user12.txt
exp26_user13.txt
exp27_user13.txt
exp28_user14.txt
exp29_user14.txt
exp30_user15.txt
exp31_user15.txt
exp32_user16.txt
exp33_user16.txt
exp34_user17.txt
exp35_user17.txt
exp36_user18.txt
exp37_user18.txt
exp38_user19.txt
exp39_user19.txt
exp40_user20.txt
exp41_user20.txt
exp42_user21.txt
exp43_user21.txt
exp44_user22.txt
exp45_user22.txt
exp46_user23.txt
exp47_user23.txt
exp48_user24.txt
exp49_user24.txt
exp50_user25.txt
exp51_user25.txt
exp52_user26.txt
exp53_user26.txt
exp54_user27.txt
exp55_user27.txt
exp56_user28.txt
exp57_user28.txt
exp58_user29.txt
exp59_user29.t

In [3]:
for i in range(N):
    print(acc[i].shape, gyro[i].shape, actions[y[i]])

(983, 3) (983, 3) STANDING
(160, 3) (160, 3) STAND_TO_SIT
(802, 3) (802, 3) SITTING
(165, 3) (165, 3) SIT_TO_STAND
(1015, 3) (1015, 3) STANDING
(288, 3) (288, 3) STAND_TO_LIE
(876, 3) (876, 3) LAYING
(197, 3) (197, 3) LIE_TO_SIT
(932, 3) (932, 3) SITTING
(192, 3) (192, 3) SIT_TO_LIE
(927, 3) (927, 3) LAYING
(191, 3) (191, 3) LIE_TO_STAND
(583, 3) (583, 3) WALKING
(895, 3) (895, 3) WALKING
(911, 3) (911, 3) WALKING
(965, 3) (965, 3) WALKING
(656, 3) (656, 3) WALKING_DOWNSTAIRS
(631, 3) (631, 3) WALKING_UPSTAIRS
(624, 3) (624, 3) WALKING_DOWNSTAIRS
(666, 3) (666, 3) WALKING_UPSTAIRS
(624, 3) (624, 3) WALKING_DOWNSTAIRS
(673, 3) (673, 3) WALKING_UPSTAIRS
(976, 3) (976, 3) STANDING
(206, 3) (206, 3) STAND_TO_SIT
(789, 3) (789, 3) SITTING
(156, 3) (156, 3) SIT_TO_STAND
(927, 3) (927, 3) STANDING
(268, 3) (268, 3) STAND_TO_LIE
(863, 3) (863, 3) LAYING
(184, 3) (184, 3) LIE_TO_SIT
(833, 3) (833, 3) SITTING
(237, 3) (237, 3) SIT_TO_LIE
(778, 3) (778, 3) LAYING
(242, 3) (242, 3) LIE_TO_STAND
(6

In [4]:
obs = np.empty((N), dtype=object)
origin_data = np.empty((N, 3), dtype=object)

def get_observation(acc, gyro):
    time = acc.shape[0]
    obs = np.empty((time), dtype=int)
    for t in range(time):
        jerkNum = accNum = gyroNum = 0
        jerk = np.zeros((3), dtype=float)
        for i in range(3):
            if t > 0:
                jerk[i] = acc[t, i] - acc[t-1, i]
                
            if jerk[i] > 0:
                jerkNum += (1 << (2-i))
            if acc[t, i] > 0:
                accNum += (1 << (2-i))
            if gyro[t, i] > 0:
                gyroNum += (1 << (2-i))
                
#         acc8vel8: 71.31147540983606%
#         obs[t] = accNum*8 + gyroNum

#         jerk8acc8: 40.98360655737705%
#         obs[t] = jerkNum*8 + accNum

#         jerk8vel8: 73.77049180327869%
        obs[t] = jerkNum*8 + gyroNum

#         jerk8angAcc8: 62.295081967213115%

#         jerk8acc2vel8: 63.11475409836065%
#         obs[t] = jerkNum*16 + gyroNum
#         if acc[t, 2] > 0:
#             obs[t] += 8
    return obs

for i in range(N):
    obs[i] = get_observation(acc[i], gyro[i])
    origin_data[i] = [y[i], obs[i], i]

print(origin_data.shape)

(1214, 3)


## train_test_split
* `train_size : test_size = 90 : 10`

In [5]:
from sklearn.model_selection import train_test_split

# train, test split
train_data, test_data = train_test_split(origin_data, test_size=0.1)

# sort in charlabel
train_data = train_data[train_data[:, 0].argsort()]
test_data = test_data[test_data[:, 0].argsort()]

print('train:', train_data.shape)
print('test:', test_data.shape)
print(test_data)

train: (1092, 3)
test: (122, 3)
[[0 array([ 5, 29, 21, ..., 48, 17, 33]) 98]
 [0
  array([ 5, 61, 63, 63, 27,  3,  3,  1, 41, 41,  5, 63, 61, 21, 21, 61, 53,
       49, 33, 32, 32,  0,  0,  0,  1, 46, 46, 62, 60, 55, 23, 31, 31, 31,
       31, 31,  3, 43, 42, 62, 38, 62, 58, 58, 58, 59, 43, 11, 11,  1, 37,
       37, 22, 30, 30, 13, 49, 33, 40, 40, 28, 20, 20, 28, 60, 60, 54, 62,
       30, 26, 10,  3, 41,  7, 47, 62, 60, 60, 44, 48, 16,  0, 32,  6, 30,
       30, 62, 54, 23, 31, 11, 11, 11, 51, 34, 50, 10, 10, 42, 58, 58, 49,
       33, 47, 27, 25, 37, 37,  7, 18, 26, 25, 33, 33, 40,  8, 24, 20, 20,
       20, 46, 14, 62, 62, 22, 22, 45, 45, 47, 39, 30, 30, 37, 36, 32,  0,
        0,  8,  0, 30, 62,  4, 52, 19,  3, 11, 11,  3, 27, 58, 50, 18, 42,
       42, 10, 14, 55, 49, 58, 42, 27, 27,  1, 37, 39, 22, 30, 26, 25, 33,
       40, 40, 44, 20, 20, 30, 30, 62, 62, 22,  4,  1, 45, 47, 47, 54, 24,
       25, 45, 37, 52,  0,  8,  8, 24, 30, 62, 52, 52, 23, 11, 11, 11, 11,
       19, 51,  2

In [6]:
idx = np.zeros(num_actions + 1, dtype=int)
for i in range(train_data.shape[0]):
    idx[train_data[i][0]+1] = i+1;

# Human Activity Recognition HMM
## HMM Learn
* `hmmlearn` Tutorial
  * https://hmmlearn.readthedocs.io/en/latest/tutorial.html
* `MultinomialHMM` API Reference
  * https://hmmlearn.readthedocs.io/en/latest/api.html#multinomialhmm

In [ ]:
models = np.empty((num_actions), dtype=object)
for i in range(num_actions):
    models[i] = hmm.MultinomialHMM(n_components=64, verbose=True, n_iter=50)

# multinomial HMM learn
very_start_time = time.time()
for i in range(num_actions):
    start_time = time.time()
    trainRange = range(idx[i], idx[i+1])
    print('Training', actions[i], 'model w.', idx[i+1]-idx[i], 'examples', end=' ')

    trainX = np.concatenate([train_data[j][1].reshape(-1, 1) for j in trainRange])
    lengths = [len(train_data[j][1]) for j in trainRange]

    models[i].fit(trainX, lengths)
    print("(elapsed time: {}s).".format(time.time() - start_time))

print("Total elapsed time: {}s.".format(time.time() - very_start_time))

Training WALKING model w. 112 examples 

         1     -446541.6401             +nan
         2     -434749.9435      +11791.6966
         3     -434725.1129         +24.8306
         4     -434699.0344         +26.0785
         5     -434670.5344         +28.4999
         6     -434638.2179         +32.3165
         7     -434600.2327         +37.9852
         8     -434553.9430         +46.2898
         9     -434495.3859         +58.5570
        10     -434418.2924         +77.0935
        11     -434312.2097        +106.0827
        12     -434158.6795        +153.5302
        13     -433922.8626        +235.8169
        14     -433533.3956        +389.4670
        15     -432827.4787        +705.9168
        16     -431371.6061       +1455.8727
        17     -427707.9332       +3663.6729
        18     -416674.9341      +11032.9991
        19     -396747.5878      +19927.3463
        20     -380589.8285      +16157.7592
        21     -369686.9629      +10902.8656
        22     -361063.6049       +8623.3580
        23

(elapsed time: 3762.9566566944122s).
Training WALKING_UPSTAIRS model w. 160 examples 

         1     -424335.7105             +nan
         2     -403469.4500      +20866.2605
         3     -403414.6591         +54.7909
         4     -403351.8773         +62.7818
         5     -403274.0479         +77.8294
         6     -403169.4035        +104.6444
         7     -403015.6759        +153.7276
         8     -402765.6079        +250.0680
         9     -402306.0465        +459.5614
        10     -401328.6195        +977.4270
        11     -398904.6966       +2423.9229
        12     -392531.6000       +6373.0966
        13     -379967.7148      +12563.8851
        14     -367033.2290      +12934.4859
        15     -354183.5697      +12849.6592
        16     -340124.7229      +14058.8469
        17     -327606.2418      +12518.4810
        18     -316747.0997      +10859.1422
        19     -308630.1211       +8116.9786
        20     -303873.0433       +4757.0778
        21     -300827.0491       +3045.9942
        22     -298332.0237       +2495.0254
        23

In [ ]:
def test_hmm_models(hmm_models):
    testSize = test_data.shape[0]
    wrongCases = 0

    print('=================== Wrong Cases ===================')
    for i in range(testSize):
        testX = np.concatenate([test_data[i][1].reshape(-1, 1)])

        maxScore = hmm_models[0].score(testX)
        maxAction = 0

        for action in range(num_actions):
            score = hmm_models[action].score(testX)
            if maxScore < score:
                maxScore = score
                maxAction = action

        if not test_data[i][0] == maxAction:
            print(actions[test_data[i][0]], actions[maxAction], maxScore, test_data[i][0] == maxAction)
            wrongCases += 1
            
            expect = origin_data[test_data[i][2]][1]
            result = origin_data[train_data[idx[maxAction]][2]][1]
            
            ax1 = plt.subplot('211')
            ax1.set_ylabel('state')
            ax1.set_title('{} vs. {}'.format(actions[test_data[i][0]], actions[maxAction]))
            plt.plot(expect)
            
            ax2 = plt.subplot('212', sharex=ax1, sharey=ax1)
            ax2.set_ylabel('state')
            ax2.set_xlabel('time')
            plt.plot(result)
            
            plt.show()

    print('(Wrong, Total)', (wrongCases, testSize))
    print('Accuracy: {}%'.format((1 - wrongCases/testSize) * 100))

In [ ]:
import pickle
import os
def save_hmm_models(models, name):
    if os.path.exists('./Models/' + name):
        os.system("rm -rf " + './Models/' + name)
    os.mkdir('./Models/' + name)
    for i in range(num_actions):
        filename = './Models/{}/{}_{:02d}_{}.pkl'.format(name, name, i, actions[i])
        with open(filename, 'wb') as file: pickle.dump(models[i], file)
        print('Save', filename)

In [ ]:
save_hmm_models(models, 'jerk8vel8iter50')

In [ ]:
import pickle
def load_hmm_models(name):
    hmm_models = []
    for i in range(num_actions):
        filename = './Models/{}/{}_{:02d}_{}.pkl'.format(name, name, i, actions[i])
        with open(filename, 'rb') as file: hmm_models.append(pickle.load(file))
        print('Open', filename)
    return hmm_models

In [ ]:
new_models = load_hmm_models('jerk8vel8iter50')
test_hmm_models(new_models)